In [29]:
!pip install captum
!pip install pytorch-lightning

DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
import torch
import pandas as pd
import numpy as np

# Import dataset utils
from torch.utils.data import Dataset, DataLoader 
from sklearn.model_selection import train_test_split

import importlib
if importlib.util.find_spec('ipywidgets') is not None:
    from tqdm.auto import tqdm
else:
    from tqdm import tqdm

In [31]:
dataframe = pd.read_csv('../data/final_novo.csv', sep=';')
dataframe.head()

,Chave,sexo,Estado_civil,Status_empl,Licenca,Tipo_Resid,Residencia,Alcoolatra,Droga,Suic_familia,...,Eixo I: Panico sem agorafobia,Eixo I: Fobia especifica,Eixo I: Fobia social,Eixo I: Obsessivo-compulsivo,Eixo I: Estresse pos-traumatico,Eixo I: Ansiedade generalizada,Eixo II: Personalidade paranoica,Eixo II: Transtorno de personalidade,TOC,idade
0,NDAR_INVLE566FK2,M,3.0,NaN,0.0,3.0,1.0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,40.0
1,NDAR_INVMA818VXP,F,1.0,3.0,0.0,4.0,3.0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,20.0
2,NDAR_INVHY103MHY,F,1.0,2.0,0.0,1.0,2.0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0
3,NDAR_INVEC849VWE,F,1.0,3.0,0.0,1.0,3.0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,30.0
4,NDAR_INVHF792VJY,F,4.0,2.0,0.0,1.0,NaN,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0


In [32]:
#último grafo cris: 8 features
selected = ["Suic_familia",
    "Capaz de tomar decisões importantes",
    "Estudante",
    "Hipocondriase",
    "Sentimentos_culpa",
    "Trabalho e interesses",
    'Dep_familia',
    'Alc_familia',
    'Capaz de desfrutar das coisas',
    'Droga',
    'Suicidio',
    'Ansiedade']


In [33]:
dataframe['sexo'].replace({'M': 0, 'F': 1}, inplace=True)
dataframe['sexo'].fillna(0, inplace=True)

df_suic = dataframe[selected]

df_suic.dropna(inplace=True)
df_suic = df_suic.astype(int)

C:\Users\Cristiane\AppData\Local\Temp\ipykernel_31000\156640811.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_suic.dropna(inplace=True)


In [34]:
df_suic

,Suic_familia,Capaz de tomar decisões importantes,Estudante,Hipocondriase,Sentimentos_culpa,Trabalho e interesses,Dep_familia,Alc_familia,Capaz de desfrutar das coisas,Droga,Suicidio,Ansiedade
0,1,2,0,2,3,2,1,1,2,0,2,2
1,1,1,0,0,2,2,1,1,1,0,1,2
2,1,2,0,0,3,3,1,1,2,0,0,2
3,1,1,0,3,4,3,1,1,1,0,3,2
4,1,2,0,1,2,4,1,1,3,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...
3948,1,2,0,0,3,2,1,1,2,0,2,3
3949,1,1,0,2,2,2,1,1,1,0,1,2
3950,1,1,0,2,3,3,1,1,1,0,2,2
3951,1,2,0,1,1,2,1,1,2,0,0,2


In [35]:
class MyDataset(Dataset):
 
  def __init__(self, input_dataframe, split="train", target="Suicidio", ignore_columns=[], train_ratio=0.8): 
    
    self.split = split
    self.target = target
    self.ignore_columns = ignore_columns

    for coll in self.ignore_columns:
       if coll in input_dataframe.columns:
        input_dataframe = input_dataframe.drop(coll, axis=1)

    self.classification_dim = len(input_dataframe[self.target].unique())
    self.data_dim = len(input_dataframe.columns) - 1
    self.embbeding_dim = input_dataframe.max().max() + 1

    y = input_dataframe[target].values 
    x = input_dataframe.drop(target, axis = 1).values 

    self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(x, y, test_size=1-train_ratio, random_state=42) 

  def __len__(self):
    if self.split == "train":
      return len(self.x_train)
    elif self.split == "test":
      return len(self.x_test)
    else:
      raise ValueError("Split must be train or test")

  def __getitem__(self,idx):
      target = torch.zeros(self.classification_dim) 
      if self.split == "train":
          target[self.y_train[idx]] = 1
          return (torch.tensor(self.x_train[idx]), target) 
      elif self.split == "test":
          target[self.y_test[idx]] = 1
          return (torch.tensor(self.x_test[idx]), target)
      else:
          raise ValueError("Split must be train or test")

In [36]:
# instanciando o dataset 
train_dataset = MyDataset(df_suic, split="train", target="Suicidio", ignore_columns=[], train_ratio=0.8)
test_dataset = MyDataset(df_suic, split="test", target="Suicidio", ignore_columns=[], train_ratio=0.8)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=False) 
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [37]:
# Define a MLP model with N layers: rede neural de 2 camadas

import torch.nn as nn
import torch.nn.functional as F

class MLP(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=128, n_layers=2):
        super(MLP, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
    
        self.layers = nn.ModuleList() 
        self.layers.append(nn.Linear(self.input_dim, self.hidden_dim)) 
        for i in range(self.n_layers - 1):
            self.layers.append(nn.Linear(self.hidden_dim, self.hidden_dim))
            self.layers.append(nn.Dropout(0.5)) 
            self.layers.append(nn.LeakyReLU())
            
        self.layers.append(nn.Linear(self.hidden_dim, self.output_dim))
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)           
        return x


In [38]:
# Define a Model with a embbeding layer and a MLP

class ClassificationModel(nn.Module):
    def __init__(self, input_dim, output_dim, embbeding_dim, hidden_out, hidden_dim=128, n_layers=2):   
        super(ClassificationModel, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.embbeding_dim = embbeding_dim
        self.embbeding_out = hidden_out
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.embbeding_layer = nn.Embedding(self.embbeding_dim, self.embbeding_out) 
        self.mlp = MLP(self.input_dim * self.embbeding_out, self.output_dim, self.hidden_dim, self.n_layers) 
        
    def forward(self, x):
        x = self.embbeding_layer(x)
        x = x.view(x.shape[0], -1)
        x = self.mlp(x)
        ## classification
        x = F.softmax(x, dim=1)
        return x

In [39]:
# test the model
example_batch = next(iter(train_loader))
example_data, example_targets = example_batch

model = ClassificationModel(train_dataset.data_dim, train_dataset.classification_dim, train_dataset.embbeding_dim, hidden_out=20, hidden_dim=128, n_layers=4) # Cadar tirou
print('model:', model, '\n')

print("Batch shape:", example_data.shape,'\n') 
res = model(example_data)
print("Output shape:", res.shape,'\n')

model: ClassificationModel(
  (embbeding_layer): Embedding(6, 20)
  (mlp): MLP(
    (layers): ModuleList(
      (0): Linear(in_features=220, out_features=128, bias=True)
      (1): Linear(in_features=128, out_features=128, bias=True)
      (2): Dropout(p=0.5, inplace=False)
      (3): LeakyReLU(negative_slope=0.01)
      (4): Linear(in_features=128, out_features=128, bias=True)
      (5): Dropout(p=0.5, inplace=False)
      (6): LeakyReLU(negative_slope=0.01)
      (7): Linear(in_features=128, out_features=128, bias=True)
      (8): Dropout(p=0.5, inplace=False)
      (9): LeakyReLU(negative_slope=0.01)
      (10): Linear(in_features=128, out_features=5, bias=True)
    )
  )
) 

Batch shape: torch.Size([128, 11]) 

Output shape: torch.Size([128, 5]) 


In [40]:
# Make Lightning Module
from pytorch_lightning import LightningModule
from torchmetrics import MaxMetric, MeanMetric
from torchmetrics.classification.accuracy import Accuracy

class BaseModel(LightningModule):
    """A LightningModule organizes your PyTorch code into 6 sections:
        - Computations (init)
        - Validation loop (validation_step)
        - Train loop (training_step)
        - Test loop (test_step)
        - Prediction Loop (predict_step)
        - Optimizers and LR Schedulers (configure_optimizers)
    """

    def __init__(self, input_dim, output_dim, embedding_dim, embedding_out, hidden_dim):
        super().__init__()
        self.model = ClassificationModel(input_dim, output_dim, embedding_dim, embedding_out, hidden_dim=hidden_dim, n_layers=2)
        self.lr = 1e-3 # taxa de aprendizado: 0.001

        self.save_hyperparameters()
        
        # metric objects for calculating and averaging accuracy across batches
        self.accuracy = Accuracy(task='multiclass', num_classes=5)
        
        # for tracking best so far validation accuracy
        self.val_acc_best = MaxMetric()
    
    def step(self, batch):
        x, y = batch
        y_hat = self.model(x).squeeze().float()
        # loss function
        loss = F.binary_cross_entropy(y_hat, y)
        acc = self.accuracy(y_hat, y.int())
        return loss, acc
   
    def training_step(self, batch, batch_idx):
        loss, acc = self.step(batch)
        self.log('train_loss', loss, prog_bar=True)        
        self.log('train_acc', acc,  prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        loss, acc = self.step(batch)
        self.log('val_loss', loss)      
        self.log('val_acc', acc,  prog_bar=True)
        return loss
    
    # gradiente
    def configure_optimizers(self):
        """Choose what optimizers and learning-rate schedulers to use in your optimization.
        Normally you'd need one. But in the case of GANs or similar you might have multiple.

        Examples:
            https://pytorch-lightning.readthedocs.io/en/latest/common/lightning_module.html#configure-optimizers
        """
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

In [41]:
# Initialize model
model = BaseModel(input_dim=train_dataset.data_dim, output_dim=train_dataset.classification_dim, embedding_dim=100, embedding_out=64, hidden_dim=128)
print('model:', model,'\n')


AssertionError: 

In [ ]:
# Import trainer
from pytorch_lightning.trainer import Trainer

In [ ]:
# Import callbacks
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

# Initialize callbacks

# Salve o modelo periodicamente monitorando uma quantidade.
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='checkpoints/', 
    filename='best-checkpoint', 
    save_top_k=1, 
    mode='min',
)

# Monitore uma métrica e interrompa o treinamento quando ela parar de melhorar.
early_stopping = EarlyStopping(
    monitor='val_loss', 
    min_delta=0.05, 
    patience=10, 
    verbose=False, 
    mode='min' 
)

callbacks = [checkpoint_callback, early_stopping]


# Initialize a trainer
trainer = Trainer(
    #accelerator='gpu',
    accelerator='cpu', 
    devices=1, 
    check_val_every_n_epoch=10, 
    log_every_n_steps=10, 
    callbacks=callbacks, 
    auto_lr_find=True, 
    enable_progress_bar=False
    ) 

In [ ]:
trainer.fit(model, train_loader, test_loader)

In [ ]:
#odel = BaseModel.load_from_checkpoint("/content/best-checkpoint-v31.ckpt")

In [ ]:
#from captum.attr import ShapleyValues
from captum.attr import ShapleyValueSampling

In [ ]:
df_suic_0 = df_suic.loc[df_suic.Suicidio == 0]
df_suic_1 = df_suic.loc[df_suic.Suicidio == 1]
df_suic_2 = df_suic.loc[df_suic.Suicidio == 2]
df_suic_3 = df_suic.loc[df_suic.Suicidio == 3]
df_suic_4 = df_suic.loc[df_suic.Suicidio == 4]

In [ ]:
svs = ShapleyValueSampling(model.model.forward)

In [ ]:
sample_df_suic_0 = df_suic_0.sample(n=10, random_state=42)
sample_df_suic_1 = df_suic_1.sample(n=10, random_state=42)
sample_df_suic_2 = df_suic_2.sample(n=10, random_state=42)
sample_df_suic_3 = df_suic_3.sample(n=10, random_state=42)
#sample_df_suic_4 = df_suic_4.sample(n=10, random_state=42)

In [ ]:
tensor_suic_0 = torch.tensor(df_suic_0.drop(labels='Suicidio', axis='columns').values)
tensor_suic_1 = torch.tensor(df_suic_1.drop(labels='Suicidio', axis='columns').values)
tensor_suic_2 = torch.tensor(df_suic_2.drop(labels='Suicidio', axis='columns').values)
tensor_suic_3 = torch.tensor(df_suic_3.drop(labels='Suicidio', axis='columns').values)
tensor_suic_4 = torch.tensor(df_suic_4.drop(labels='Suicidio', axis='columns').values)

In [ ]:
attr_tensor_suic_0 = svs.attribute(tensor_suic_0, target=0)
attr_tensor_suic_1 = svs.attribute(tensor_suic_1, target=1)
attr_tensor_suic_2 = svs.attribute(tensor_suic_2, target=2)
attr_tensor_suic_3 = svs.attribute(tensor_suic_3, target=3)
attr_tensor_suic_4 = svs.attribute(tensor_suic_4, target=4)

In [ ]:
feature_names = list(df_suic.drop(labels='Suicidio', axis='columns').columns)
feature_names

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
# Helper method to print importances and visualize distribution
def visualize_importances(feature_names, importances, title="Average Feature Importances", plot=True):
    print(title)
    for i in range(len(feature_names)):
        print(feature_names[i], ": ", '%.3f'%(importances[i]))
    x_pos = (np.arange(len(feature_names)))
    if plot:
        plt.figure(figsize=(12,6))
        plt.bar(x_pos, importances, align='center')
        plt.xticks(x_pos, feature_names, wrap=True)
        plt.xticks(rotation=20, ha='right')
        plt.xlabel("Features")
        plt.ylabel("Average Feature Importances")
        plt.grid(True)
        plt.title(title)

In [ ]:
attr_np_0 = attr_tensor_suic_0.detach().numpy()
attr_np_1 = attr_tensor_suic_1.detach().numpy()
attr_np_2 = attr_tensor_suic_2.detach().numpy()
attr_np_3 = attr_tensor_suic_3.detach().numpy()
attr_np_4 = attr_tensor_suic_4.detach().numpy()

In [ ]:
visualize_importances(feature_names, np.mean(attr_np_0, axis=0), title="Classe Suícidio = 0")
plt.savefig("shap_suicidio_0.png")

In [ ]:
visualize_importances(feature_names, np.mean(attr_np_1, axis=0), title="Classe Suícidio = 1")
plt.savefig("shap_suicidio_1.png")

In [ ]:
visualize_importances(feature_names, np.mean(attr_np_2, axis=0), title="Classe Suícidio = 2")
plt.savefig("shap_suicidio_2.png")

In [ ]:
visualize_importances(feature_names, np.mean(attr_np_3, axis=0), title="Classe Suícidio = 3")
plt.savefig("shap_suicidio_3.png")

In [ ]:
visualize_importances(feature_names, np.mean(attr_np_4, axis=0), title="Classe Suícidio = 4")
plt.savefig("shap_suicidio_4.png")